In [959]:
###################################################################################################
# Implementation of clustering aggregation algorithms i) Neighbors ii) Hierarchical iii) KwikSort
# Input: People with features, size k
# Output: Clusters of size close to k
# Method: Clustering Aggregation
# Problem Formulation: Given clusterings C1,C2,...,Cm split the aggreement graph into parts of size close to k
# (within some approximation) such that you maximize the sum of the weights of the edges within the clusters.
# The weight of edge e={u,v} is the fraction of clusterings that put u and v in the same cluster
# - maximizes the agreement.
# Purpose: Creating teams for hackthegap
###################################################################################################

In [2]:
import itertools as it
import pandas as pd
import math
import numpy as np
import networkx as nx
import random
import itertools
import math

from itertools import chain
from itertools import izip
# from constrained_kmeans import KMeans
from itertools import combinations
from sklearn.metrics import jaccard_similarity_score as JS
from sklearn.metrics import mean_squared_error
from math import sqrt

In [3]:
# Input: user ids
# Output: Undirected graph where each user is a node and each user has an edge with each other user
def initGraph(features):
    users = list(features.keys())
    # makes graph complete
    edges = combinations(users, 2)
    
    # creating the graph
    user_graph=nx.Graph()
    user_graph.add_nodes_from(users)
    user_graph.add_edges_from(edges)
    
    return user_graph

In [4]:
def weight_computation(weight_metric,features_user1,features_user2):
    
    weight = float("inf")
    if weight_metric == 'difference':
        weight = 1 - (abs(features_user1 - features_user2)/float(2))
    elif weight_metric == 'jaccard':
        intersect = len(list(set(features_user1) & set(features_user2)))
        union = len(list(set(features_user1) | set(features_user2)))
        weight = intersect / float(union)
    elif weight_metric == 'same':
        if isinstance(features_user1,int):
            if features_user1 == features_user2:
                weight = 1
            else:
                weight = 0
        else:
            idx1 = features_user1.index(1)
            idx2 = features_user2.index(1)
            if idx1 == idx2:
                weight = 1
            else:
                weight = 0
            
    return weight      

In [5]:
# Input: feature vectors
# Output: weighted clusterings based on features, mapping between user pair and weights in different clusterings
# Note!! 0: NumberOfHackathons, 1-6: Skills, 7: Beginner, 8-14: Interests. These are the distinct different features
# that lead to the corresponding number of clusterings i.e. 4

def create_clusterings(features,user_graph):
    
    clusterings = []
    clustering_weights = {}
    
    # indexes that show where each feature ends, for instance
    # feature skill can contain several values whose indexes range from 1-6
    indexes = [0,6,7,14]
    weight_metrics = ['difference','jaccard','same','same']
    
    # creating the weights for each clustering  - all weights range from 0 to 1
    # numhackathons clustering (difference): weight = 1 - (|num1 - num2|/2)
    # skills clustering (jaccard): Jaccard(set1,set2)
    # beginner clustering (same): 1 if both beginners, or if both non-beginners, otherwise 0
    # interest (same): 1 if same interest, otherwise 0
    
    counter = 0; values = []
    for idx, value in enumerate(indexes):
        user_graph_copy = nx.Graph(user_graph)
        
        weight_metric = weight_metrics[idx]
        
        for user1,user2 in user_graph_copy.edges(data=False):
            key = (user1,user2)
            # print 'Key is:',key
            if idx == 0 or (indexes[idx] - indexes[idx-1]) == 1: 
                features_user1 = features[user1][idx]
                features_user2 = features[user2][idx]
                weight = weight_computation(weight_metric,features_user1,features_user2)
                user_graph_copy.edge[user1][user2]['weight'] = weight
            else:
                features_user1 = features[user1][indexes[idx-1]+1:indexes[idx]+1]
                features_user2 = features[user2][indexes[idx-1]+1:indexes[idx]+1]
                weight = weight_computation(weight_metric,features_user1,features_user2)
                user_graph_copy.edge[user1][user2]['weight'] = weight
                
            if key in clustering_weights:
                clustering_weights[key].append(weight)
            else:
                clustering_weights[key] = [];
                clustering_weights[key].append(weight)
                              
        clusterings.append(user_graph_copy)
        
    return clusterings, clustering_weights    

In [6]:
# Aggrement_Graph: Creating the aggreement graph used by the proposed team formation algorithms 
# Input: data
# Output: Aggreement Graph 
# Note!! 0: NumberOfHackathons, 1-6: Skills, 7: Beginner, 8-14: Interests. These are the distinct different features
def aggreement_graph(features):
    print 'Creating aggreement graph'
    # creating initial unweighted graph
    user_graph = initGraph(features)
    aggreement_graph = nx.Graph(user_graph)
    
    (clusterings, clustering_weights) = create_clusterings(features,user_graph)
    for key,value in clustering_weights.iteritems():
        user1 = key[0]; user2 = key[1]
        value[1] = value[1]*2
        value[2] = value[2]*5
        value[3] = value[3]*10
        
        weight = sum(value)/float(len(value)+2+5+10)
        aggreement_graph.edge[user1][user2]['weight'] = weight
    
    for u in aggreement_graph.nodes():
        aggreement_graph.node[u]['size'] = 1
    
    return aggreement_graph

In [7]:
def merge_nodes(u,v,Merge_AGraph,algo):
    size = float("inf")
    new_node = ()
    # 1) create a new cluster node with a) updated cluster nodes (u+v), b) updated cluster size
    if algo == 'kwiksort':
        if isinstance(v,tuple):
            size = len(v) + 1
            new_node = (v,u)
            new_node = tuple(list(new_node[0]) + [new_node[1]])
        else:
            size = 2
            new_node = (v,u)
            # print '2 new node:',new_node
            
    elif algo == 'hierarchical':
        new_node = (v,u)
        new_node = tuple(chain(*(i if isinstance(i, tuple) else (i,) for i in new_node)))
        size = len(new_node)
        
    # adding new node to graph    
    Merge_AGraph.add_node(new_node)
    Merge_AGraph.node[new_node]['size'] = size
    # 2) for each neighbor of u and v compute average weight edge - Should not matter 
    # if it is u or v because it is a complete graph

    neighbors_u = Merge_AGraph.neighbors(u)
    
    for neighbor in neighbors_u:
        flag = False
        if Merge_AGraph.has_edge(u,neighbor):
            weight_u = Merge_AGraph.edge[u][neighbor]['weight']
        else:
            flag = True
        if Merge_AGraph.has_edge(neighbor,v):    
            weight_v = Merge_AGraph.edge[neighbor][v]['weight']
        else:
            flag = True

        if flag == False:
            new_weight = (weight_u+weight_v)/float(2)
            Merge_AGraph.add_edge(new_node,neighbor) 
            Merge_AGraph.edge[new_node][neighbor]['weight'] = new_weight

    
    # 3) remove old nodes u,v
    Merge_AGraph.remove_node(v)
    Merge_AGraph.remove_node(u)
    return Merge_AGraph

In [8]:
# Hierarchical
# Input: Aggreement Graph, k
# Output: m clusters
def constrained_hierarchical(AGraph,k): 
    Copy_AGraph = nx.Graph(AGraph)
    removed_nodes = {}
    weight_flag = False
    while Copy_AGraph.edges and weight_flag == False:
        max_edge_weight = -float("inf")
        for (u,v) in Copy_AGraph.edges(data=False):
            size1 = Copy_AGraph.node[u]['size']
            size2 = Copy_AGraph.node[v]['size']
            weight = Copy_AGraph.edge[u][v]['weight']
            if weight > max_edge_weight and size1+size2 <= k:
                new_edge = (u,v)
                max_edge_weight = weight
        
        u = new_edge[0]; v = new_edge[1]
        if max_edge_weight != -float("inf"):
            Copy_AGraph  = merge_nodes(u,v,Copy_AGraph,'hierarchical')
            for (u,data) in Copy_AGraph.nodes(data=True):
                size = data['size']
                if size == k:
                    neighbors = Copy_AGraph.edges(u,data=True)
                    removed_nodes[u] = neighbors

        else:
            weight_flag = True
    return Copy_AGraph.nodes(data=False)

In [9]:
# Hierarchical
# Input: Aggreement Graph, k
# Output: m clusters
def unconstrained_hierarchical(AGraph,k): 
    Copy_AGraph = nx.Graph(AGraph)
    removed_nodes = {}
    weight_flag = False
    while Copy_AGraph.edges and weight_flag == False:
        max_edge_weight = -float("inf")
        for (u,v) in Copy_AGraph.edges(data=False):
            size1 = Copy_AGraph.node[u]['size']
            size2 = Copy_AGraph.node[v]['size']
            weight = Copy_AGraph.edge[u][v]['weight']
            if weight > max_edge_weight:
                new_edge = (u,v)
                max_edge_weight = weight
        
        u = new_edge[0]; v = new_edge[1]
        if max_edge_weight != -float("inf") and max_edge_weight >= 0.5:
            Copy_AGraph  = merge_nodes(u,v,Copy_AGraph,'hierarchical')
            for (u,data) in Copy_AGraph.nodes(data=True):
                size = data['size']
                if size == k:
                    neighbors = Copy_AGraph.edges(u,data=True)
                    removed_nodes[u] = neighbors

        else:
            weight_flag = True
    return Copy_AGraph.nodes(data=False)

In [10]:
# Neighbors 
# Input: Aggreement Graph, k
# Output: m clusters 

# TODO
def neighbors(AGraph,k):
    print 'Running neighbors algorithm'    

In [11]:
# KwikSort
# Input: Aggreement Graph, k
# Output: m clusters
def kwiksort(AGraph,k,person_ids,pairs,num_iter,threshold):
    
    solution = []

    return solution

In [12]:
def write_to_output(solution,features,idx_RMSD,idx_a,idx_s,key_names,output_file):
    data = []; names_team = []
    counter = 1
    # print 'Solution in write to output:',solution
    max_cluster = -float("inf")
    for cluster in solution:
        if len(cluster) > max_cluster:
            max_cluster = len(cluster)

    for idx,cluster in enumerate(solution):
        feature_list = []; row = {}; names_team = []
        for person in cluster:
            name = key_names[person]
            names_team.append(name)
            features_per_person = features[person]
            name
            feature_list.append(features_per_person)
            
        feature_list += [''] * (max_cluster - len(feature_list)+1)
        row['cluster'] = counter  
        row['cluster size'] = len(cluster)
        row['person ids'] = cluster
        row['names'] = names_team
        row['Avg RMSD'] = idx_RMSD[idx]
        row['Avg A'] = idx_a[idx]
        row['Avg S'] = idx_s[idx]
        for i in range(max_cluster+1):
            column_value = 'Member '+str(i)
            row[column_value] = feature_list[i]
            
        data.append(row)   
        counter+=1
        
    output_solution = pd.DataFrame(data=data)
    person_ids = output_solution['person ids']; cluster_size = output_solution['cluster size']; \
    cluster = output_solution['cluster']
    output_solution.drop(labels=['person ids','cluster size','cluster'], axis=1,inplace = True)
    output_solution.insert(0, 'cluster', cluster)
    output_solution.insert(1, 'cluster size', cluster_size)
    output_solution.insert(2, 'person ids', person_ids)
    
    output_solution.to_csv(output_file, index=False)

In [13]:
def findsubsets(features,m):
    person_ids = list(features.keys())
    return list(itertools.combinations(person_ids, m))

In [14]:
def assign_small_clusters(min_team_size,max_team_size,solution,features,interests):
    new_solution = []
    distribute = {}; remove_cluster = []
    # creating a distribution of all clusters and 
    for idx,cluster in enumerate(solution):
        if len(cluster) < min_team_size:
            for person in cluster:
                features_per_person = features[person]
                index = features_per_person[8:15].index(1)
                interest = interests.keys()[interests.values().index(index)] 
                distribute[person] = interest
            remove_cluster.append(idx)
            
    for idx in remove_cluster:
        del solution[idx]
    
    for idx,cluster in enumerate(solution):
        if len(cluster) < max_team_size and idx not in remove_cluster:
            for person in cluster:
                features_per_person = features[person]
                index = features_per_person[8:15].index(1)
                interest = interests.keys()[interests.values().index(index)]
                for key,value in distribute.iteritems():
                    if interest == 'I have no idea' or interest == value and len(cluster) < max_team_size:
                        cluster = cluster + (key,)
                        del distribute[key]
                        break  
        new_solution.append(cluster)
            
                       
    return new_solution
                

In [80]:
# constrained_KwikSort
# Input: Aggreement Graph, k, threshold as percentile
# Output: m clusters

def constrained_kwiksort(AGraph,k,threshold):
    solution = [] 
    while len(solution)==0:
        # print('try kwiksort')
        solution = kwiksort_helper(AGraph.copy(),k,threshold)
    print('feasible solution found')
    return solution

# unconstrained_KwikSort
# Input: Aggreement Graph, k, threshold
# Output: m clusters
def unconstrained_kwiksort(AGraph,k,threshold):
    # solution should be a list of lists or tuples
    solution= []
    
    ## Fill in here
    while (AGraph.number_of_nodes()>k):
            
        u = AGraph.nodes()[random.randint(0,AGraph.number_of_nodes()-1)]
        neighbors = AGraph.neighbors(u)
        weights = {}
        for neighbor in neighbors:
            weights[neighbor] = AGraph.edge[u][neighbor]['weight']
        weights = sorted(weights.iteritems(), key=lambda (k,v): (v,k))[::-1]
        
        cluster = []
        for i in range(len(weights)):
            if weights[i][1] > threshold:
                cluster.append(weights[i][0])  
        if cluster != []:
            cluster.append(u)
            solution.append(cluster)
            AGraph.remove_nodes_from(cluster)
            
    if AGraph.number_of_nodes()>0: 
        solution.append(AGraph.nodes())
    return solution

# helper function
def kwiksort_helper(AGraph,k,threshold):
    # solution should be a list of lists or tuples
    solution= []
    fail = 0 
    percentile = threshold
    
    ## Fill in here
    while (AGraph.number_of_nodes()>k):
        # dynamic threshold (70%tile)
        dist = []
        for edge in AGraph.edges_iter():
            dist.append(AGraph.edge[edge[0]][edge[1]]['weight'])
        threshold = np.percentile(dist, percentile)
        
        # sort neighbors according to weights 
        u = AGraph.nodes()[random.randint(0,AGraph.number_of_nodes()-1)]
        neighbors = AGraph.neighbors(u)
        weights = {}
        for neighbor in neighbors:
            weights[neighbor] = AGraph.edge[u][neighbor]['weight']
        weights = sorted(weights.iteritems(), key=lambda (k,v): (v,k))[::-1]

        # if found enough similar neighbors, form a cluster; o.w. choose another node
        if weights[k-2][1] > threshold:
            cluster = [x[0] for x in weights[:k-1]] + [u]
            solution.append(cluster)
            AGraph.remove_nodes_from(cluster)
        else: 
            fail = fail+ 1
            if fail > 20: return [] 
    
    if AGraph.number_of_nodes()>0: 
        solution.append(AGraph.nodes())
    return solution


In [88]:
def CK_threshold_test(input_file,team_size,min_team_size,num_iter,threshold):
    (data, features, person_features,unique,key_names) = data_preprocessing(input_file)
    pairs = findsubsets(features,2)
    person_ids = list(features.keys())
    AGraph = aggreement_graph(features)  
    
    print '*'*50
    con_kwik_rmsd = []
    con_kwik_a = []; con_kwik_s = []; con_kwik_clusters_a = []; con_kwik_clusters_s = []
    for j in range(70,95,5):
        print 'threshold:',j, 'th percentile'
        for i in range(6):
            IAGraph = nx.Graph(AGraph)
#             print 'Running constrained kwiksort for time:',i
            solution = constrained_kwiksort(IAGraph,team_size,i)
#             print 'Computing Root Mean Square Deviation'
            (score_RMSD,clusters_RMSD,idx_RMSD_CK) = rmsd(solution,features)
            con_kwik_rmsd.append(score_RMSD)
#             print 'Computing Silhouette Index'
            (a_score,s_score,clusters_a,clusters_s,idx_a_CK,idx_s_CK) = silhouette_index(solution,features)
            con_kwik_a.append(a_score); con_kwik_s.append(s_score) 
            con_kwik_clusters_a.append(clusters_a); con_kwik_clusters_s.append(clusters_s)
        score_RMSD = sum(con_kwik_rmsd)/float(len(con_kwik_rmsd))
        a_score = sum(con_kwik_a)/float(len(con_kwik_a)); s_score = sum(con_kwik_s)/float(len(con_kwik_s))
        print 'Root Mean Square Deviation for constrained kwiksort is:',score_RMSD
        print 'A score for constrained kwiksort is:',a_score
        print 'S score for constrained kwiksort is:',s_score

In [16]:
# Computes the centroid from a list of data points
def mean(a):
    return sum(a) / float(len(a))

def centroid(l):
    c = map(mean, zip(*l))  
    return c

In [17]:
def pairwise(t):
    pairs = list(itertools.combinations(t, 2))
    return pairs

In [68]:
def rmsd(solution,features):
    score = 0
    # print 'Solution:',solution
    # print 'Features:',features
    rmsds = []; clusters_rmsd = []; idx_rmsd = {}
    
    for idx,team in enumerate(solution):
        team_features = []; rmsds = []
        for member in team:
            team_features.append(features[member])
        team_features = np.array(team_features)
        team_features = team_features[:,7:15].tolist()
        # team_features = team_features.tolist()
        c = centroid(team_features)   
        # print 'Centroid is:',c
        for person in team_features:
            # print 'Person is:',person
            # print 'MSE:',mean_squared_error(person, c)
            rms = sqrt(mean_squared_error(person, c))
            # print 'RMS is:',rms
            rmsds.append(rms)
            
        avg_per_cluster = sum(rmsds)/float(len(rmsds))
        idx_rmsd[idx] = avg_per_cluster
        # print 'Average cluster is:',avg_per_cluster
        clusters_rmsd.append(avg_per_cluster)
        # print
    
    
    score = sum(clusters_rmsd)/float(len(clusters_rmsd))
    # print 'RMSD Score:',score
    return score,clusters_rmsd,idx_rmsd

In [19]:
def silhouette_index(solution,features):
    score = 0
    a_precomp = {}; a = {}; avg_per_cluster = []; a_avg_of_clusters = []; s_avg_of_clusters = []
    b_precomp = {}; b = {}; rms_per_user = []
    s = {};
    team_members = {}; non_team_members = {};
    idx_a = {}; idx_b = {}; idx_s = {}
    members = []; users = []
    
    # list of all users and team members
    for idx,team in enumerate(solution):
        for member in team:
            users.append(member)
            copy_team = list(team)
            copy_team.remove(member)
            remaining_team = copy_team
            team_members[member] = remaining_team
            
    # list of non-team members
    for member in users:
        team = team_members[member]
        non_team = list(set(users) - set(team))
        copy_team = list(non_team)
        copy_team.remove(member)
        non_team_members[member] = copy_team

    # computing a - compactness
    for idx,team in enumerate(solution):
        pairs = pairwise(team)  
        sum_sed = 0
        for pair in pairs:
            u1 = pair[0]; x = features[u1];
            u2 = pair[1]; y = features[u2];
            rms = sqrt(mean_squared_error(x[7:15], y[7:15]))
            # rms = sqrt(mean_squared_error(x, y))
            if u1 in a_precomp:
                a_precomp[u1].append(rms)
            else:
                a_precomp[u1] = []
                a_precomp[u1].append(rms)
                
            if u2 in a_precomp:
                a_precomp[u2].append(rms)
            else:
                a_precomp[u2] = []
                a_precomp[u2].append(rms)  
        
        for key,value in a_precomp.iteritems():
            avg_per_user = sum(value)/float(len(team) - 1)
            # a(i)
            a[key] = avg_per_user
            avg_per_cluster.append(avg_per_user)
            
        avg_cluster = sum(avg_per_cluster)/float(len(avg_per_cluster))
        idx_a[idx] = avg_cluster
        a_avg_of_clusters.append(avg_cluster)
        a_precomp = {}; avg_per_cluster = []
    
    a_score = sum(a_avg_of_clusters)/float(len(a_avg_of_clusters))
    
    # computing b - separation
    for key,value in non_team_members.iteritems():
        x = features[key]
        for non_member in value:
            y = features[non_member]
            # rms = sqrt(mean_squared_error(x[7:15], y[7:15]))
            rms = sqrt(mean_squared_error(x, y))
            rms_per_user.append(rms)

        min_dist = min(rms_per_user)
        b[key] = min_dist
        rms_per_user = []

    for key, value in a.iteritems():
        # You want a to be small (distance from same cluster points)
        # You want b to be large (distance from different cluster points)
        if b[key]!=a[key]:
            s[key] = (b[key] - a[key])/float(max(b[key],a[key]))
        else:
            s[key] = 0 
    
    s_cluster = []; avg_clusters = []
    for idx,team in enumerate(solution):
        for member in team:
            s_cluster.append(s[member])

        cluster_avg = sum(s_cluster)/float(len(s_cluster))
        idx_s[idx] = cluster_avg
        s_avg_of_clusters.append(cluster_avg)
        s_cluster = []
    
    s_score = sum(s_avg_of_clusters)/float(len(s_avg_of_clusters))
    return (a_score,s_score,a_avg_of_clusters,s_avg_of_clusters,idx_a,idx_s)

In [20]:
def test():
    test_features = {}
    test_features[1] = [2,1,1,3]; test_features[2] = [1,0,0,1]; test_features[5] = [1,0,0,1]
    test_features[3] = [1,0,0,1]; test_features[4] = [1,1,0,0]
    team1 = (1,2,5); team2 = (3,4)
    test_solution = []; test_solution.append(team1); test_solution.append(team2)
    (score_RMSD,clusters_RMSD) = rmsd(test_solution,test_features)
    print 'Root Mean Square Deviation for constrained hierarchical is:',score_RMSD
    (a_score,s_score,clusters_a,clusters_s) = silhouette_index(test_solution,test_features)
    print 'A score for constrained hierarchical is:',a_score
    print 'S score for constrained hierarchical is:',s_score
    print 'A score cluster averages for constrained hierarchical is:',clusters_a
    print 'S score cluster averages for constrained hierarchical is:',clusters_s
    print 'Finished with test'    

In [29]:
def data_preprocessing(input_file):
    key_names = {}
    data = pd.read_csv(input_file)
    data = data.dropna(subset=['#'])
    data = data.reset_index(drop=True)
    # Changing features/skill to binary
    data['Beginner'] = data['Beginner'].map({'Beginner': 1, None: 0})
    data['Python'] = data['Python'].map({'Python': 1, None: 0})
    data['HTML5/CSS'] = data['HTML5/CSS'].map({'HTML5/CSS': 1, None: 0})
    data['Javascript'] = data['Javascript'].map({'Javascript': 1, None: 0})
    data['Java'] = data['Java'].map({'Java': 1, None: 0})
    data['SQL'] = data['SQL'].map({'SQL': 1, None: 0})
    data['C++'] = data['C++'].map({'C++': 1, None: 0})
    
    # Renaming columns
    data_names = data.rename(columns=\
                       {'What type of technology are you interested in learning during the hackathon?': 'Technology',\
                        'How many hackathons have you participated in before?': 'NumHackathons',\
                        'What\'s your first name?':'FirstName',\
                        'Hey {{answer_57551534}}, nice to meet you.What\'s your last name?':'LastName',\
                        '#': 'Id'})
    
    # Dropping columns not needed
    data = data_names[['Id','NumHackathons','Python','HTML5/CSS','Javascript','Java','SQL','C++','Beginner','Technology']]
    technology_list = data['Technology'].tolist()
    unique_technologies = list(set(technology_list))
    
    # Dictionary with key: person id and value: dictionary of features
    person_data = {}; skills = []
    for index, row in data.iterrows():
        skills = []
        key = row['Id'];
        first_name = data_names.at[index, 'FirstName']
        last_name = data_names.at[index, 'LastName']

        num_hackathons = row['NumHackathons']; 
        if row['Python']==1:
            skills.append('Python') 
        if row['HTML5/CSS']==1:
            skills.append('HTML5/CSS')
        if row['Javascript']==1:
            skills.append('Javascript')
        if row['Java']==1:
            skills.append('Java')
        if row['SQL']==1:
            skills.append('SQL')
        if row['C++']==1:
            skills.append('C++')
        beginner = 'Beginner' if row['Beginner']==1 else 'Non-Beginner'; technology = row['Technology']
        value = {'num_hackathons':num_hackathons,'skills':skills,'experience':beginner,'technology':technology}
        person_data[key] = value
        key_names[key] = str(first_name)+' '+str(last_name)
        
    unique_technologies_dict = {}

    counter = 0
    for technique in unique_technologies:
        unique_technologies_dict[technique] = counter
        counter+=1
    
    set_of_technologies = []
    for technology in technology_list:
        listofzeros = [0] * len(unique_technologies)
        index = unique_technologies_dict[technology]
        listofzeros[index] = 1
        set_of_technologies.append(listofzeros)

    # Extracting features 3) Number of hackathos 4) Skills * 6 5) Beginner
    #                     6) Technology 
    # create dictionary of person : [features]
    
    features = \
            {data.loc[idx, 'Id']: [data.loc[idx, 'NumHackathons'], data.loc[idx, 'Python'], data.loc[idx, 'HTML5/CSS'],\
                                   data.loc[idx, 'Javascript'], data.loc[idx, 'Java'], data.loc[idx, 'SQL'],\
                                   data.loc[idx, 'C++'], data.loc[idx, 'Beginner']] \
                                   for idx in range(data.shape[0])}
    
    counter = 0
    for key,value in features.iteritems():
        value.extend(set_of_technologies[counter])
        counter+=1
        
    return data, features, person_data, unique_technologies_dict,key_names

In [107]:
def main(input_file,team_size,min_team_size,num_iter,threshold):
    (data, features, person_features,unique,key_names) = data_preprocessing(input_file)
    pairs = findsubsets(features,2)
    person_ids = list(features.keys())
    AGraph = aggreement_graph(features)  
    
    # IAGraph = nx.Graph(AGraph)
    # print 'Running kwiksort'
    # solution = kwiksort(IAGraph,team_size,person_ids,pairs,num_iter,threshold)
    # write_to_output(solution,person_features,'RecommendedTeamsKwik.csv')
    
    
    # creating test sample for rmsd
    # test()

    IAGraph = nx.Graph(AGraph)
    print 'Running constrained hierarchical'
    solution = constrained_hierarchical(IAGraph,team_size)
    # print 'Solution:',solution
    solution = assign_small_clusters(min_team_size,team_size,solution,features,unique)
    print 'Computing Root Mean Square Deviation'
    (score_RMSD,clusters_RMSD,idx_RMSD_CH) = rmsd(solution,features)
    print 'Root Mean Square Deviation for constrained hierarchical is:',score_RMSD
    print 'Computing Silhouette Index'
    (a_score,s_score,clusters_a,clusters_s,idx_a_CH,idx_s_CH) = silhouette_index(solution,features)
    print 'A score for constrained hierarchical is:',a_score
    print 'S score for constrained hierarchical is:',s_score
    print 'A score cluster averages for constrained hierarchical is:',clusters_a
    print 'S score cluster averages for constrained hierarchical is:',clusters_s
    # print 'Writing to file'
    write_to_output(solution,person_features,idx_RMSD_CH,idx_a_CH,idx_s_CH,key_names,'ConstrainedHierarch.csv')
    
    print '*'*50
    IAGraph = nx.Graph(AGraph)
    print 'Running unconstrained hierarchical'
    solution = unconstrained_hierarchical(IAGraph,team_size)
    # print 'Solution:',solution
    solution = assign_small_clusters(min_team_size,team_size,solution,features,unique)
    print 'Computing Root Mean Square Deviation'
    (score_RMSD,clusters_RMSD,idx_RMSD_UH) = rmsd(solution,features)
    print 'Root Mean Square Deviation for unconstrained hierarchical is:',score_RMSD
    print 'Computing Silhouette Index'
    (a_score,s_score,clusters_a,clusters_b,idx_a_UH,idx_s_UH) = silhouette_index(solution,features)
    print 'A score for unconstrained hierarchical is:',a_score
    print 'S score for unconstrained hierarchical is:',s_score
    print 'A score cluster averages for unconstrained hierarchical is:',clusters_a
    print 'S score cluster averages for unconstrained hierarchical is:',clusters_s
    # print 'Writing to file'
    write_to_output(solution,person_features,idx_RMSD_UH,idx_a_UH,idx_s_UH,key_names,'UnconstrainedHierarch.csv')
    
    print '*'*50
    con_kwik_rmsd = []
    con_kwik_a = []; con_kwik_s = []; con_kwik_clusters_a = []; con_kwik_clusters_s = []
    for i in range(6):
        IAGraph = nx.Graph(AGraph)
        print 'Running constrained kwiksort for time:',i
        solution = constrained_kwiksort(IAGraph,team_size,70)
        '''temperary: offset the cluster with size one 
        
        '''
        solution = solution[:-1]
        print 'Computing Root Mean Square Deviation'
        (score_RMSD,clusters_RMSD,idx_RMSD_CK) = rmsd(solution,features)
        con_kwik_rmsd.append(score_RMSD)
        print 'Computing Silhouette Index'
        (a_score,s_score,clusters_a,clusters_s,idx_a_CK,idx_s_CK) = silhouette_index(solution,features)
        con_kwik_a.append(a_score); con_kwik_s.append(s_score) 
        con_kwik_clusters_a.append(clusters_a); con_kwik_clusters_s.append(clusters_s)
    
    score_RMSD = sum(con_kwik_rmsd)/float(len(con_kwik_rmsd))
    a_score = sum(con_kwik_a)/float(len(con_kwik_a)); s_score = sum(con_kwik_s)/float(len(con_kwik_s))
    print 'Root Mean Square Deviation for constrained kwiksort is:',score_RMSD
    print 'A score for constrained kwiksort is:',a_score
    print 'S score for constrained kwiksort is:',s_score
    print 'A score cluster average indicative for constrained kwiksort is:',clusters_a
    print 'S score cluster average indicative for constrained kwiksort is:',clusters_s
    # print 'Solution:',solution
    write_to_output(solution,person_features,idx_RMSD_CK,idx_a_CK,idx_s_CK,key_names,'ConstrainedKwiksort.csv')
        
    print '*'*50
    uncon_kwik_rmsd = []
    uncon_kwik_a = []; uncon_kwik_s = []; uncon_kwik_clusters_a = []; uncon_kwik_clusters_s = []
    for i in range(2):
        IAGraph = nx.Graph(AGraph)
        print 'Running unconstrained kwiksort'
        solution = unconstrained_kwiksort(IAGraph,team_size,1/float(2))
#         print 'Solution:',solution
        
        '''temperary: offset the cluster with size one 
        
        '''
        solution = [x for x in solution if len(x)>1]

        print 'Computing Root Mean Square Deviation'
        (score_RMSD,clusters_RMSD,idx_RMSD_UK) = rmsd(solution,features)
        uncon_kwik_rmsd.append(score_RMSD)
        print 'Computing Silhouette Index'
        (a_score,s_score,clusters_a,clusters_s,idx_a_UK,idx_s_UK) = silhouette_index(solution,features)
        uncon_kwik_a.append(a_score); uncon_kwik_s.append(s_score) 
        uncon_kwik_clusters_a.append(clusters_a); uncon_kwik_clusters_s.append(clusters_s)
    
    score_RMSD = sum(uncon_kwik_rmsd)/float(len(uncon_kwik_rmsd))
    a_score = sum(uncon_kwik_a)/float(len(uncon_kwik_a)); s_score = sum(uncon_kwik_s)/float(len(uncon_kwik_s))
    print 'Root Mean Square Deviation for unconstrained hierarchical is:',score_RMSD
    print 'A score for unconstrained kwiksort is:',a_score
    print 'S score for unconstrained kwiksort is:',s_score
    print 'A score cluster average indicative for unconstrained kwiksort is:',clusters_a
    print 'S score cluster average indicative for unconstrained kwiksort is:',clusters_s
    write_to_output(solution,person_features,idx_RMSD_UK,idx_a_UK,idx_s_UK,key_names,'UnconstrainedKwiksort.csv')


In [108]:
# Team constraints: 1) Each team is a team of approximately 5 2) Similar users and interests are grouped together 
input_file = 'Hackthegap_clean_final#.csv'
dataset_file = 'dataset.csv'
num_iter = 1
threshold = 1/float(2)
# Removing first empty row from dataset -- TODO
with open(input_file,'r') as f, open(dataset_file,'w') as f1:
    # next(f) # skip header line
    for line in f:
        f1.write(line)
        
data = pd.read_csv(dataset_file)    

max_team_size = 5
min_team_size = 3
main(dataset_file, max_team_size, min_team_size, num_iter, threshold)

Creating aggreement graph
Running constrained hierarchical
Computing Root Mean Square Deviation
Root Mean Square Deviation for constrained hierarchical is: 0.0703661074948
Computing Silhouette Index
A score for constrained hierarchical is: 0.091947386172
S score for constrained hierarchical is: 0.217264756669
A score cluster averages for constrained hierarchical is: [0.23570226039551587, 0.23570226039551587, 0.1767766952966369, 0.0, 0.0, 0.0, 0.0, 0.14142135623730953, 0.0, 0.1767766952966369, 0.0, 0.1414213562373095, 0.0, 0.0, 0.21213203435596428, 0.21213203435596428, 0.14142135623730953, 0.0, 0.14142135623730953, 0.0, 0.3, 0.14142135623730953, 0.0, 0.1767766952966369, 0.14142135623730953, 0.0, 0.0, 0.0]
S score cluster averages for constrained hierarchical is: [-0.21763301114443737, 0.4719565586661787, 0.15767340155927104, 1.0, 1.0, 0.25, 0.25, -0.5224059710201014, 0.25, -1.0, 1.0, 0.3261387212474169, 1.0, 0.0, 0.17869967831924338, 0.024059935816290512, -1.0, 1.0, 0.5123041256365672, 